## ⚖️ Count Number of Balanced Permutations

---

### ✅ 1. Problem Statement

Given a string `num` containing only digits, return the number of **balanced permutations** of its digits.

A permutation is **balanced** if the sum of digits at **even indices** is equal to the sum of digits at **odd indices**.

---

### 💡 2. Approach

This problem is a variation of the **subset-sum problem with position constraints**, and we solve it using **recursive memoization with combinatorics**:

1. Convert the string `num` to a list of digits.
2. Check if the total sum of digits is even. If not, no balanced permutation is possible.
3. Use a recursive function to:
   - Select how many of each digit to place in even and odd indices.
   - Track how much of the target sum is left for even positions.
   - Use combinatorics to count **ways to assign digits** to those positions (with `choose`).
4. Memoize the recursive function with `@cache`.

In [1]:
### 💻 3. Python Code (with Detailed Inline Comments)

from typing import List
from functools import cache
from collections import Counter

def countBalancedPermutations(num: str) -> int:
    # Convert string to list of integers
    digits = [int(digit) for digit in num]

    # Count how many times each digit appears
    counter = Counter(digits)

    total_sum = sum(digits)
    n = len(digits)
    mod = 10**9 + 7

    # If total sum is odd, we can't divide it equally into two halves
    if total_sum % 2 != 0:
        return 0

    target = total_sum // 2  # We want both sides to sum to this
    even_slots = n // 2      # Number of even index positions
    odd_slots = (n + 1) // 2 # Number of odd index positions

    # Combinatorics: Compute C(n, k) % mod using Pascal's rule
    @cache
    def choose(n, k):
        if k == 0 or k == n:
            return 1
        if k < 0 or k > n:
            return 0
        return (choose(n - 1, k - 1) + choose(n - 1, k)) % mod

    # Recursive function to try placing digits from current digit onward
    @cache
    def dfs(digit, target, even_left, odd_left):
        # If all positions filled and target sum achieved on even side
        if target == 0 and even_left == 0 and odd_left == 0:
            return 1

        # Out of bounds or invalid state
        if digit > 9 or target < 0 or even_left < 0 or odd_left < 0:
            return 0

        count = counter.get(digit, 0)  # Number of occurrences of current digit
        ways = 0

        # Try all possible ways to assign `pick` copies to even indices
        for pick in range(min(count, even_left) + 1):
            odd_pick = count - pick  # Remaining copies go to odd indices
            if odd_pick > odd_left:
                continue  # Skip if we don't have enough odd positions

            # Recursively solve for next digit
            partial = dfs(
                digit + 1,
                target - pick * digit,     # Reduce target sum accordingly
                even_left - pick,          # Reduce even slots used
                odd_left - odd_pick        # Reduce odd slots used
            )

            # Ways to choose positions for `pick` digits in even slots
            ways_to_place_even = choose(even_left, pick)

            # Ways to choose positions for remaining digits in odd slots
            ways_to_place_odd = choose(odd_left, odd_pick)

            # Multiply possibilities and add to total ways
            total_ways = partial * ways_to_place_even * ways_to_place_odd
            ways = (ways + total_ways) % mod

        return ways

    # Start recursion with digit 0
    return dfs(0, target, even_slots, odd_slots)

In [2]:
### 🧪 4. Example Calls

print(countBalancedPermutations("123"))     # Output: 2
print(countBalancedPermutations("112"))     # Output: 1
print(countBalancedPermutations("12345"))   # Output: 0

2
1
0


### 📊 5. Complexity

- **Time Complexity**:  
  `O(10 * T * E * O)` where:
  - 10: max number of digits (0 to 9),
  - T = target sum (at most 9 * n / 2),
  - E = even_slots,
  - O = odd_slots  
  Due to memoization, this is efficient for `n ≤ 10`.

- **Space Complexity**:  
  `O(T * E * O * 10)` for cache + recursion stack.

### 🧠 6. Insight

This problem cleverly combines:
- Subset sum with restricted positions (even/odd index).
- Counting permutations with repeated elements.
- Efficient memoization via state compression.